In [ ]:
import os
from __future__ import print_function

master = '--master local[1]'
#master = '--master spark://apachespark-master-2-1-0:7077'
jars = '--jars /root/lib/jpmml-sparkml-1.2-SNAPSHOT.jar'

packages = ''
  #'--packages org.jpmml:jpmml-sparkml:1.2.2'

py_files = '--py-files /root/lib/jpmml.py'

os.environ['PYSPARK_SUBMIT_ARGS'] = master \
  + ' ' + jars \
  + ' ' + packages \
  + ' ' + py_files \
  + ' ' + 'pyspark-shell'

print(os.environ['PYSPARK_SUBMIT_ARGS'])

In [ ]:
from pyspark.ml import Pipeline

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler


In [ ]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [ ]:
import pandas as pd
from sklearn import datasets
iris = datasets.load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)

df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
df['species'] = pd.factorize(df['species'])[0]

df.columns = ["sepal_length(cm)", \
              "sepal_width(cm)", \
              "petal_length(cm)", \
              "petal_width(cm)", \
              "species"]

df_iris = sqlContext.createDataFrame(df)

df_iris.printSchema()

df_iris.show(5)

#train = df_iris

train, test = df_iris.randomSplit([0.7, 0.3])

train.count()
test.count()

In [ ]:
from pyspark.ml.classification import LogisticRegression

from pyspark.ml import Pipeline

assembler = VectorAssembler(inputCols=["sepal_length(cm)", \
                                       "sepal_width(cm)", \
                                       "petal_length(cm)", \
                                       "petal_width(cm)"], 
                            outputCol="features")

lr = LogisticRegression(featuresCol='features', labelCol='species')

pipeline = Pipeline(stages=[assembler, lr])

pipeline_model = pipeline.fit(train)

print(pipeline_model)

In [ ]:
lr_model = pipeline_model.stages[1]
print(lr_model.numClasses)

In [ ]:
train.schema

In [ ]:
prediction_lr = pipeline_model.transform(test)

score_lr = prediction_lr.select(['species', 'prediction'])
score_lr.show(n=score_lr.count())

prediction_lr.show(5)

In [ ]:
from jpmml import toPMMLBytes

In [ ]:
pmmlBytes = toPMMLBytes(sc, train, pipeline_model)
print(pmmlBytes.decode("UTF-8"))